In [240]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## Importing Analytical Report and calculating data

In [241]:
df = pd.read_excel('0FLATANALYTICALexport.XLSX')

In [242]:
#correction of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'][df['Due date (-)'] == '1816-03-29 05:56:08.066277376' ] = 0
df['test'] = np.where(df['Due date (-)'] == '1970-01-01', 0, 1)

# correction of Title transfer date (-)
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype(object).where(df['Title transfer date (-)'].notnull(), 0)
df['Title transfer date (-)'] = np.where(df['Title transfer date (-)'] == 0, pd.to_datetime('today') + np.timedelta64(30,'D') , df['Title transfer date (-)'])
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype('datetime64[ns]')

#continuation of correction of Due date (-)
df['Due date (-) corrected'] = np.where(df['test'] == 1, df['Due date (-)'], df['Due date (+)'] - df['Title transfer date (+)'] + df['Title transfer date (-)'])

df['Due date (-)'] = df['Due date (-) corrected']

# converting the type of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'] = df['Due date (-)'].dt.date

df['Due date (+)'] = df['Due date (+)'].dt.date
df['Title transfer date (-)'] = df['Title transfer date (-)'].dt.date
df['Title transfer date (+)'] = df['Title transfer date (+)'].dt.date


#filling NaN
df['Incoterms 1 (-)'].fillna(df['Incoterms 1 (+)'], inplace=True)
df['Incoterms 2 country (-)'].fillna(df['Incoterms 2 country (+).1'], inplace=True)
df['Incoterms 2 country (-).1'].fillna(df['Incoterms 2 country (+)'], inplace=True)


In [243]:
# calculating the columns

df['Consumption Date'] = df['Due date (+)']
df['Consumption Type'] = 'Filled by Macros'
df['Total WC Consumption period'] = df['Due date (-)'] - df['Due date (+)']
df['TTD difference'] = df['Title transfer date (-)'] - df['Title transfer date (+)']
df['Sale Payment Terms'] = df['Due date (-)'] - df['Title transfer date (-)']
df['Buy Payment Terms'] = df['Due date (+)'] - df['Title transfer date (+)']
df['PDD difference'] = df['Sale Payment Terms'] - df['Buy Payment Terms']
df['Consumption reason'] = 'OTHER'
df['WC Amount'] = df['WC consumption amount'] / 1000000
df['Current amount in USD (+)'] = df['Quantity BBL (ratio) (+)'] * df['Current price, USD/BBL (+)']
df['Current amount in USD (-)'] = df['Quantity BBL (ratio) (-)'] * df['Current price, USD/BBL (-)']

df['Company Name'] = df['Line of Business'].str[:3]

df['Portfolio name change'] = df['Company Name'] + ' ' + df['Portfolio name']
df['Portfolio name'] = df['Portfolio name change']


# converting columns to the right format
#converting into days
df['Total WC Consumption period'] = df['Total WC Consumption period'].dt.days
df['TTD difference'] = df['TTD difference'].dt.days
df['Sale Payment Terms'] = df['Sale Payment Terms'].dt.days
df['Buy Payment Terms'] = df['Buy Payment Terms'].dt.days
df['PDD difference'] = df['PDD difference'].dt.days

#fixing the counterparty where Litasco appears
df['Counterparty (+)'] = np.where(df['Counterparty (+)'] == 'LITASCO SA', df['Company Name'] + ' ' + df['Mirror portfolio name (+)'], 
                                  df['Counterparty (+)'])

df['Counterparty (-)'].fillna('', inplace=True)
df['Counterparty (-)'] = np.where(df['Counterparty (-)'] == 'LITASCO SA', df['Company Name'] + ' ' + df['Mirror portfolio name (-)'], 
                                  df['Counterparty (-)'])


In [244]:
# correcting the WC Amount
df_strategy = pd.DataFrame()
df_strategy = df[['Strategy', 'WC Amount','Strategy start date']].copy()


#import the strategies
df_strategy_exclude = pd.read_excel('Strategies.xlsx')

#conditions and sum
df_strategy = df_strategy.merge(df_strategy_exclude, on = 'Strategy', how = 'left')
df_strategy['New'].fillna(0, inplace=True)

df_strategy['S2'] = np.where(df_strategy['Strategy'] > 517000, 1,0)
df_strategy['ssd'] = np.where(df_strategy['Strategy start date'] > '2015-12-01', 1,0)

df_strategy['sum_s'] = df_strategy['New'] + df_strategy['ssd'] + df_strategy['S2']

df_strategy['WC Amount corrected'] = np.where(df_strategy['sum_s'] > 0,df_strategy['WC Amount'], 0)

# replacing WC amount
df['WC Amount'] = df_strategy['WC Amount corrected']


In [245]:
def f(row):
    if (row['Total WC Consumption period'] >= row['TTD difference'] and row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] <= row['TTD difference'] and row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['TTD difference']
    elif (row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['Total WC Consumption period']
    else:
        return 0

In [246]:
df['TTD influence on WC consumption'] = df.apply(f, axis=1)
df['PDD influence on WC consumption'] = df['Total WC Consumption period'] - df['TTD influence on WC consumption']


In [247]:
# rearrange columns
cols = df.columns.tolist()

In [248]:
cols = ['Consumption Date',
       'Consumption Type',
       'Total WC Consumption period',
        'TTD influence on WC consumption',
        'PDD influence on WC consumption',
        'TTD difference',
        'PDD difference',
        'Sale Payment Terms',
        'Buy Payment Terms',
        'Consumption reason',
        'WC Amount',
        'Buy/Sell (-)',
        'Buy/Sell (+)',
        'Due date (-)',
        'Due date (+)',
        'Title transfer date (-)',
        'Title transfer date (+)',
        'Strategy',
        'Strategy name',
        'Company Name',
        'Global Book name',
        'Line of Business',
        'Book Name',
        'Portfolio name',
        'Quantity',
        'Current amount in USD (+)',
        'Quantity BBL (ratio) (+)',
        'Current price, USD/BBL (+)',
        'Counterparty (+)',
        'Title transfer (+)',
        'Incoterms 1 (+)',
        'Incoterms 2 country (+)',
        'Incoterms 2 country (+).1',
        'Object type (+)',
        'Delivery ID (+)',
        'Internal type (+)',
        'Mirror Delivery ID (+)',
        'Current amount in USD (-)',
        'Quantity BBL (ratio) (-)',
        'Current price, USD/BBL (-)',
        'Counterparty (-)',
        'Title transfer (-)',
        'Incoterms 1 (-)',
        'Incoterms 2 country (-)',
        'Incoterms 2 country (-).1',
        'Object type (-)',
        'Delivery ID (-)',
        'Internal type (-)',
        'Mirror Delivery ID (-)',
        'Strategy start date',
        'Strategy end date',
        'Voyage ID (+)',
        'Transport name (+)',
        'Grade (+)',
        'Voyage ID (-)',
        'Transport name (-)',
        'Grade (-)']

## Next steps:
1. remove rows for which we have zeros in WC Amount
2. remove rows for which we have time < - 365 and > 365

## Exporting the file

In [249]:
# export

df_export = df[cols]

# export to excel
writer = pd.ExcelWriter('output.xlsx')
df_export.to_excel(writer,'Sheet1')
writer.save()

In [250]:
# ****************** columns to correct***************************
# counterparty (+), some differences in method. not sure if it adds value changing...
# counterparty (-), some differences in method. not sure if it adds value changing...


## Next steps:
1. remove rows for which we have zeros in WC Amount
2. remove rows for which we have time < - 365 and > 365
3. create Stock in Transit (new df)
4. create Payment Terms (new df)
5. aggregate Stock in Transit and Payment Terms (new df)
6. actuals
7. others
8. adjustments